In [1]:
# paquetes necesarios

import csv # para escribir .csv
import pandas as pd # para manejo de estructuras pandas DataFrame
import os  # para leer .csv
import time # se importa sleep para pausar el código en tiempo controlado
from time import sleep
from getpass import getpass #escribir contraseña oculta
# Paquetes para utilizar navegador
from msedge.selenium_tools import Edge, EdgeOptions  
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.select import Select
from selenium.common import exceptions

In [2]:
# crea la instancia para abrir el webbrowser MS EDGE
def create_webdriver_instance():
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options)
    return driver

# Ingresa a la pagina de twitter, ingresa credenciales, si falla devuelve error y detiene la operación. Entra y luego va a la pestaña home
def login_to_twitter(username, password, driver):
    url = 'https://twitter.com/login'
    try:
        driver.get(url)
        xpath_username = '//input[@name="session[username_or_email]"]'
        WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, xpath_username)))
        uid_input = driver.find_element_by_xpath(xpath_username)
        uid_input.send_keys(username)
    except exceptions.TimeoutException:
        print("Timeout while waiting for Login screen")
        return False

    pwd_input = driver.find_element_by_xpath('//input[@name="session[password]"]')
    pwd_input.send_keys(password)
    try:
        pwd_input.send_keys(Keys.RETURN)
        url = "https://twitter.com/home"
        WebDriverWait(driver, 10).until(expected_conditions.url_to_be(url))
    except exceptions.TimeoutException:
        print("Timeout while waiting for home screen")
    return True

#ingresa el término de busqueda a la casilla de búsqueda de twitter

## !!!!!!!!!! si twitter esta en ingles el área de búsqueda tiene un valor de "Search query"
## pero en español tiene un valor de "Búsqueda"
def find_search_input_and_enter_criteria(search_term, driver):
    xpath_search = '//input[@aria-label="Search query"]'  # Inglés: Search query, Españo : Búsqueda
    search_input = driver.find_element_by_xpath(xpath_search)
    search_input.send_keys(search_term)
    search_input.send_keys(Keys.RETURN)
    return True


# cliquea la opcion "tab_name", se utiliza para ir a la opción "Latest", de los tweets 
def change_page_sort(tab_name, driver):
    """Options for this program are `Latest` and `Top`"""
    tab = driver.find_element_by_link_text(tab_name)
    tab.click()
    xpath_tab_state = f'//a[contains(text(),\"{tab_name}\") and @aria-selected=\"true\"]'

# genera id de tweet para identificalo y ver que no se repita  
def generate_tweet_id(tweet):
    return ''.join(tweet)

# operación de scrolling retiene la última posición para comparar si se movió o no y así detener la operación
# tiene los parámetros de tiempo que espera para realizar el scroll ( para que cargue la página)
# max_attempts es el número de intentos de scrolling para detener la operación
# cuando exede el número de intentos se detiene, esto para cuando se llegue al final de la cadena de búsqueda
#  num_seconds_to_load es el tiempo que espera para realizar el scroll, entonces si carga rápido la pagina se puede reducir, si tarda en cargar se puede aumentar
def scroll_down_page(driver, last_position, num_seconds_to_load=2, scroll_attempt=0, max_attempts=5):
    """The function will try to scroll down the page and will check the current
    and last positions as an indicator. If the current and last positions are the same after `max_attempts`
    the assumption is that the end of the scroll region has been reached and the `end_of_scroll_region`
    flag will be returned as `True`"""
    end_of_scroll_region = False
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # (0 , document.body.scrollHeight) son las coordenadas para mover la barra de scroll
    sleep(num_seconds_to_load)
    curr_position = driver.execute_script("return window.pageYOffset;")
    if curr_position == last_position:
        if scroll_attempt < max_attempts:
            end_of_scroll_region = True
        else:
            scroll_down_page(last_position, curr_position, scroll_attempt + 1) # se compara la posición para ver si se está moviendo o no, de forma de determinar si se llega al final
    last_position = curr_position
    return last_position, end_of_scroll_region

#guarda el tweet
def save_tweet_data_to_csv(records, filepath, mode='a+'):
    header = ['User', 'Handle', 'PostDate', 'TweetText', 'ReplyCount', 'RetweetCount', 'LikeCount']
    with open(filepath, mode=mode, newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if mode == 'w':
            writer.writerow(header)
        if records:
            writer.writerow(records)
# extrae todos los tweets que hay cargados en la página, para despues realizar el scroll e iterar nuevamente
# twitter carga dinámicamente los tweets en el código html, entonces para que sean cargados a la página se debe
# realizar el scroll, de otra forma no se cargan.
def collect_all_tweets_from_current_view(driver, lookback_limit=25):
    """The page is continously loaded, so as you scroll down the number of tweets returned by this function will
     continue to grow. To limit the risk of 're-processing' the same tweet over and over again, you can set the
     `lookback_limit` to only process the last `x` number of tweets extracted from the page in each iteration.
     You may need to play around with this number to get something that works for you. I've set the default
     based on my computer settings and internet speed, etc..."""
    page_cards = driver.find_elements_by_xpath('//div[@data-testid="tweet"]')
    if len(page_cards) <= lookback_limit:
        return page_cards
    else:
        return page_cards[-lookback_limit:]

# extrae la información del tweet
def extract_data_from_current_tweet_card(card):
    try:
        user = card.find_element_by_xpath('.//span').text
    except exceptions.NoSuchElementException:
        user = ""
    except exceptions.StaleElementReferenceException:
        return
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except exceptions.NoSuchElementException:
        handle = ""
    try:
        """
        If there is no post date here, there it is usually sponsored content, or some
        other form of content where post dates do not apply. You can set a default value
        for the postdate on Exception if you which to keep this record. By default I am
        excluding these.
        """
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except exceptions.NoSuchElementException:
        return
    try:
        _comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    except exceptions.NoSuchElementException:
        _comment = ""
    try:
        _responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    except exceptions.NoSuchElementException:
        _responding = ""
    tweet_text = _comment + _responding
    try:
        reply_count = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    except exceptions.NoSuchElementException:
        reply_count = ""
    try:
        retweet_count = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    except exceptions.NoSuchElementException:
        retweet_count = ""
    try:
        like_count = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    except exceptions.NoSuchElementException:
        like_count = ""

    tweet = (user, handle, postdate, tweet_text, reply_count, retweet_count, like_count) # agregar reply_to
    return tweet

In [3]:
# CREDENCIALES
url = 'https://twitter.com/login'

username = "usuario123"            # nombre de usuario o correo
password = "clave123"                 # contraseña
filepath = '#EstásEstamos.csv'        # nombre de archivo de extracción (el nombre del .csv que se guarda)
search_term = '#EstásEstamos'         # termino de busqueda ( se ingresa textual sin " " en la barra de búsqueda)
page_sort = "Latest"  

**1. Iniciar navegador**: se inicia la sesión de EDGE a través de Python

In [4]:
driver = create_webdriver_instance()

In [5]:
driver.maximize_window()

In [6]:
driver.get(url)

In [7]:
username = "gabriel.vergara@sansano.usm.cl"

In [8]:
xpath_username = '//input[@name="session[username_or_email]"]'
uid_input = driver.find_element_by_xpath(xpath_username)
uid_input.send_keys(username)

In [9]:
password = getpass()

 ··············


In [10]:
pwd_input = driver.find_element_by_xpath('//input[@name="session[password]"]')
pwd_input.send_keys(password)

In [11]:
pwd_input.send_keys(Keys.RETURN)

In [28]:
#login_to_twitter(username, password, driver)

True

**Campos de búsqueda avanzada**

**1) Campo: Words**

In [35]:
driver.get("https://twitter.com/search-advanced")

In [31]:
All_of_these_words = " "
xpath = '//input[@name="allOfTheseWords"]'
uid_input = driver.find_element_by_xpath(xpath)
uid_input.send_keys(All_of_these_words)

In [33]:
This_exact_phrase = " "
xpath = '//input[@name="thisExactPhrase"]'
uid_input = driver.find_element_by_xpath(xpath)
uid_input.send_keys(This_exact_phrase)

In [13]:
Any_of_these_words = "bancodechile bancochile bancosantander santanderchile bancobci bsantander bancoedwards bedwards banchile"
xpath = '//input[@name="anyOfTheseWords"]'
uid_input = driver.find_element_by_xpath(xpath)
uid_input.send_keys(Any_of_these_words)

In [37]:
None_of_these_words = " "
xpath = '//input[@name="noneOfTheseWords"]'
uid_input = driver.find_element_by_xpath(xpath)
uid_input.send_keys(None_of_these_words)

In [14]:
these_Hashtags = "#santanderchile #bancochile #estasestamos #bancoestado #bancobci"
xpath = '//input[@name="theseHashtags"]'
uid_input = driver.find_element_by_xpath(xpath)
uid_input.send_keys(these_Hashtags )

In [36]:
select_option = "Any language"
xpath = "Language"
uid_input = driver.find_element_by_id(xpath)
select = Select(uid_input)
select.select_by_visible_text(select_option)

**2) Campo: Accounts**

In [37]:
from_these_accounts = "@latercera"
xpath = '//input[@name="fromTheseAccounts"]'
uid_input = driver.find_element_by_xpath(xpath)
uid_input.send_keys(from_these_accounts)

In [43]:
to_these_accounts = " "
xpath = '//input[@name="toTheseAccounts"]'
uid_input = driver.find_element_by_xpath(xpath)
uid_input.send_keys(to_these_accounts)

In [44]:
mentioning_these_accounts = " "
xpath = '//input[@name="mentioningTheseAccounts"]'
uid_input = driver.find_element_by_xpath(xpath)
uid_input.send_keys(mentioning_these_accounts)

**3) Campo: Dates**

In [38]:
from_year = "2006"
xpath = '/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div[2]/div/div/div/div[16]/div/div[2]/div/div[3]/select'
uid_input = driver.find_element_by_xpath(xpath)
select = Select(uid_input)
select.select_by_visible_text(from_year)

In [39]:
to_year = "2021"
xpath = '/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div[2]/div/div/div/div[16]/div/div[4]/div/div[3]/select'
uid_input = driver.find_element_by_xpath(xpath)
select = Select(uid_input)
select.select_by_visible_text(to_year)

**Finalizar y ejecutar el filtro**

In [40]:
tab_name = "Search"
xpath = "/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div[1]/div/div/div/div[3]/div/div/span"
driver.find_element_by_xpath(xpath).click()

**Ir a la tab Latest**

In [41]:
change_page_sort("Latest", driver)

**Algoritmo de copia de tweets**

In [42]:
save_tweet_data_to_csv(None, filepath, 'w')  # create file for saving records
last_position = None
end_of_scroll_region = False
unique_tweets = set()

In [45]:
while not end_of_scroll_region:
    cards = collect_all_tweets_from_current_view(driver)
    for card in cards:
        try:
            tweet = extract_data_from_current_tweet_card(card)
        except exceptions.StaleElementReferenceException:
            continue
        if not tweet:
            continue
        tweet_id = generate_tweet_id(tweet)
        if tweet_id not in unique_tweets:
            unique_tweets.add(tweet_id)
            save_tweet_data_to_csv(tweet, filepath)
    last_position, end_of_scroll_region = scroll_down_page(driver, last_position,num_seconds_to_load=0.5)

KeyboardInterrupt: 